In [ ]:
!pip install cloudinary python-dotenv transformers accelerate fastapi uvicorn pyngrok peft==0.15.0
!pip install git+https://github.com/huggingface/diffusers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 11.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.3/147.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [ ]:
!ngrok config add-authtoken <ngrok token>


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from io import BytesIO
import cloudinary
import cloudinary.uploader
from peft import PeftModel 
from PIL import Image
import os
from dotenv import load_dotenv

load_dotenv()

base_model = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
).to("cuda")

lora_path = "/kaggle/input/skibidii/pytorch/default/1/pytorch_lora_weights.safetensors"  # Replace with your path
base_model.load_lora_weights(lora_path)

def gen_image(prompt: str) -> str:
    image = base_model(prompt).images[0]

    buffer = BytesIO()
    image.save(buffer, format="PNG")
    buffer.seek(0)

    cloudinary.config( 
        cloud_name='', 
        api_key='', 
        api_secret= "", 
        secure=True
    )

    upload_result = cloudinary.uploader.upload(
        buffer,
        folder="lora_outputs",       
        format="png"
    )

    print("Image uploaded to Cloudinary!")
    return upload_result['secure_url']


2025-05-29 06:52:55.543348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748501575.985801      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748501576.102988      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from diffusers import StableDiffusionInstructPix2PixPipeline
import torch
from PIL import Image
import requests
from io import BytesIO
import cloudinary
import cloudinary.uploader
import os
from dotenv import load_dotenv
load_dotenv()
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
        "timbrooks/instruct-pix2pix",
        torch_dtype=torch.float16,
        safety_checker=None
    ).to("cuda")
def edit_image(cloudinary_url, prompt):
    response = requests.get(cloudinary_url)
    if response.status_code != 200 or "image" not in response.headers["Content-Type"]:
        raise Exception("Invalid image URL")

    input_image = Image.open(BytesIO(response.content)).convert("RGB")

    output = pipe(prompt, image=input_image)
    generated_image = output.images[0]

    buffer = BytesIO()
    generated_image.save(buffer, format="PNG")
    buffer.seek(0)

    cloudinary.config( 
        cloud_name='', 
        api_key='', 
        api_secret= "", 
        secure=True
    )
    upload_result = cloudinary.uploader.upload(
        buffer,
        folder="lora_outputs",           
        format="png"
    )

    return upload_result['secure_url']


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# import functions
#from gen_model import gen_image
#from edit_model import edit_image

# Initialize
app = FastAPI()
nest_asyncio.apply()

class GenRequest(BaseModel):
    prompt: str

class EditRequest(BaseModel):
    prompt: str
    url: str

@app.post("/generate")
def generate(data: GenRequest):
    url = gen_image(data.prompt)
    return {"url": url}

@app.post("/edit")
def edit(data: EditRequest):
    url = edit_image(data.url, data.prompt)
    return {"url": url}

# Tunnel to public URL
public_url = ngrok.connect(7860)
print("Public URL:", public_url)

# Run the app
uvicorn.run(app, port=7860)


ModuleNotFoundError: No module named 'fastapi'